Karamcheti, S., Nair, S., Balakrishna, A., Liang, P., Kollar, T., & Sadigh, D. (2024). [Prismatic VLMs: Investigating the Design Space of Visually-Conditioned Language Models](https://arxiv.org/abs/2402.07865). arXiv preprint arXiv:2402.07865.

**Abstract**
<!-- Visually-conditioned language models (VLMs) have seen growing adoption in applications such as visual dialogue, scene understanding, and robotic task planning; adoption that has fueled a wealth of new models such as LLaVa, InstructBLIP, and PaLI-3. Despite the volume of new releases, key design decisions around image preprocessing, architecture, and optimization are under-explored, making it challenging to understand what factors account for model performance − a challenge further complicated by the lack of objective, consistent evaluations. To address these gaps, we first compile a suite of standardized evaluations spanning visual question answering, object localization, and challenge sets that probe properties such as hallucination; evaluations that provide fine-grained insight VLM capabilities. Second, we rigorously investigate VLMs along key design axes, including pretrained visual representations and training from base vs. instruct-tuned language models, amongst others. We couple our analysis with three resource contributions: (1) a unified framework for evaluating VLMs, (2) optimized, flexible training code, and (3) checkpoints for all models, including a family of VLMs at the 7-13B scale that strictly outperform InstructBLIP and LLaVa v1.5, the state-of-the-art in open VLMs. -->
视觉条件语言模型 (VLMs) 已在视觉对话、场景理解和机器人任务规划等应用程序中得到越来越广泛的采用; 这种应用催生了大量新模型, 例如 LLaVa、InstructBLIP 和 PaLI-3。尽管新发布数量众多, 但围绕图像预处理、架构和优化的关键设计决策尚未得到充分探索, 使得理解哪些因素影响模型性能变得具有挑战性——缺乏客观、一致的评估使这一挑战更加复杂。为了解决这些差距, 我们首先编制了一套标准化评估, 涵盖视觉问答、物体定位和探索幻觉等属性的挑战集; 评估提供细粒度洞察的 VLM 功能。其次, 我们沿关键设计轴严格调查 VLM, 包括预训练的视觉表征和从基础语言模型与指令调整语言模型进行的训练等。我们将我们的分析与三个资源贡献相结合: (1) 用于评估 VLMs 的统一框架, (2) 优化的、灵活的训练代码, (3) 所有模型的检查点, 包括 7-13B 规模的 VLMs 系列, 其严格优于开源 VLMs 中最先进的 InstructBLIP 和 LLaVa v1.5。

# Introduction
<!-- Visually-conditioned language models (VLMs) generate natural language responses from image input and text prompts, providing a general, expressive interface for a growing spectrum of applications – grounded chat (Li et al., 2023c; Gong et al., 2023), visual programming (Sur'is et al., 2023; Subramanian et al., 2023), robotic control (Driess et al., 2023; Brohan et al., 2023), etc. This broad adoption is fueled by a recent paradigm shift in how we develop VLMs; eschewing the complex architectures and training objectives of prior work (Tan & Bansal, 2019; Li et al., 2022; 2023b), new VLMs adopt a simple approach, treating patch features from pretrained visual backbones (e.g., CLIP; Radford et al., 2021) as tokens that can be projected into the input space of a language model (LM). This "patch-as-token" approach enables training with a simple objective – next-token prediction – and allows us to harness the ecosystem of powerful LMs such as Llama-2 and Mistral (Touvron et al., 2023; Jiang et al., 2023), along with the tools to efficiently train them (e.g., FSDP; Zhao et al., 2023). This combination has fueled the rapid development and release of models such as LLaVa v1.5, and PALI-3 that adopt the same underlying recipe, while varying individual ingredients such as the choice of pretrained components, data, or optimization procedure (Liu et al., 2023b; Chen et al., 2023b). -->
视觉条件语言模型 (VLMs) 从图像输入和文本提示生成自然语言响应, 为日益广泛的应用提供通用且富有表现力的接口——基础聊天 (Li et al., 2023c; Gong et al., 2023)、可视化编程 (Sur'is et al., 2023; Subramanian et al., 2023)、机器人控制 (Driess et al., 2023; Brohan et al., 2023) 等。最近我们开发 VLM 方式的范式转变推动了这种广泛采用; 避开先前工作的复杂架构和训练目标 (Tan & Bansal, 2019; Li et al., 2022; 2023b), 新的 VLMs 采用简单的方法, 将预训练视觉 backbones (e.g., CLIP; Radford et al., 2021) 的 patch 特征视为词元, 它可以被投影到语言模型 (LM) 的输入空间。这种"patch 即词元"方法能够以简单的目标(下一个词元预测)进行训练, 并允许我们利用强大的 LMs (例如 Llama-2 和 Mistral)的生态系统 (Touvron et al., 2023; Jiang et al., 2023), 以及有效训练它们的工具(例如 FSDP; Zhao et al., 2023)。这种结合推动了 LLaVa v1.5 和 PALI-3 等模型的快速开发和发布, 这些模型采用相同的底层方法, 同时改变各个成分, 例如预训练组件、数据或优化程序的选择 (Liu et al., 2023b; Chen et al., 2023b)。

Unfortunately, existing approaches only cover a sliver of the design space around building and training VLMs, without thoroughly evaluating the impact of given choices on downstream capabilities. This motivates the key question of this work: what are the key design decisions that influence VLM capabilities and downstream use? To provide answers to this question, we first need a way to thoroughly evaluate the strengths and weaknesses of a given model. Doing this effectively requires compiling a standardized evaluation suite comprised of tasks that are diverse and objective; crucially, these tasks should allow for probing specific capabilities such as spatial reasoning, out-of-distribution generalization, and commonsense understanding, amongst others. Second, we need to rigorously explore different VLM design axes, not only to build a concrete set of recommendations, but to tie individual choices to downstream performance.


This work addresses these axes through four contributions. First, to provide fine-grained insight into VLM capabilities, we compile a standardized evaluation suite comprised of twelve benchmarks from the vision-and-language literature, including four tasks spanning visual question answering (Bigham et al., 2010; Goyal et al., 2017; Hudson & Manning, 2019; Singh et al., 2019), four tasks spanning object localization (Kazemzadeh et al., 2014; Yu et al., 2016; Wang et al., 2021), and four challenge tasks evaluating fine-grained spatial reasoning, hallucination, and diagram understanding (Acharya et al., 2018; Liu et al., 2022; Li et al., 2023d; Kembhavi et al., 2016). Second, we develop an optimized and modular codebase for VLM training that emphasizes flexibility, allowing users to easily swap in pretrained components, optimization procedures, data, and more (Fig. 2; right). Third, we use these resource contributions to perform targeted experiments exploring four key design axes (Fig. 2; left): 1) optimization procedure, 2) image processing and visual representations, 3) language models, and 4) scaling training time and data. We identify a number of insights; for example, we find that multi-stage training procedures adopted by existing work can be eliminated without impact on performance, reducing compute costs by 20-25%. We also find that fused visual backbones that merge features from different backbones such as CLIP (Radford et al., 2021) and DINOv2 (Oquab et al., 2023) lead to more performant VLMs across the board. Finally, we consolidate our findings and train a family of models – PRISMs – at the 7B/13B scale that strictly outperform state-of-the-art open VLMs such as InstructBLIP and LLaVa v1.5.